In [1]:
with open('requirements.txt', 'w') as f:
    f.write("""pandas>=1.5.0
numpy>=1.21.0
scikit-learn>=1.0.0
matplotlib>=3.5.0
seaborn>=0.11.0
xgboost>=1.7.6
yfinance>=0.2.0
sklearn>=1.6.1            
statsmodels>=0.13.0""")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mltester import run_mltester # remove?
import sys
import os
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, mean_squared_error
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.dummy import DummyRegressor
from __future__ import annotations
from pathlib import Path
from typing import List, Optional, Tuple, Type

# Python 3.13.7


## NOTE; HOW TO RUN CODE/NOTEBOOK FOR ASSESSOR:

**Use of Artificial Intelligence Acknowledgement: Some Files within this assignment have been created with the assistance of Microsoft Copilot - indicated by a top comment**

### Option 1: Estimated Completion Time: 
If you wish to run the notebook, you can do so via the "Run All" button on the very top.

-----------------------------------------------------


### Option 2: Estimated Completion Time: 
If you wish to run only the required code; run the first two Python cells in this notebook and search (Ctrl+F) in this notebook for **%run DataPreparation/KMeans_Data_Prep_robustScaler.py** and run the cell, then search for **%run student_ext.py** and run this cell. 
Alternatively, you can run them via terminal commands (ensure you are running from project root):

**via macOS/Linux:**
#### Option A: Run with the default py 
python DataPreparation/KMeans_Data_Prep_robustScaler.py
python student_ext.py

#### Option B: If your default is Python 2, use python3 explicitly
python3 DataPreparation/KMeans_Data_Prep_robustScaler.py
python3 student_ext.py



**via PowerShell/CommandPrompt**:
#### PowerShell
python .\DataPreparation\KMeans_Data_Prep_robustScaler.py
python .\student_ext.py

#### If needed, explicitly use py launcher to select Python 3
py -3 .\DataPreparation\KMeans_Data_Prep_robustScaler.py
py -3 .\student_ext.py

-----------------------------------------------------

To save on time, I have included my part 1 results, my original clustering model, experimentations of my clustering model, and baseline comparison as .csv files and I have imported their findings into this notebook. 


# Part 2: Multivariate Regime Detection and Predictive Modelling

Noted suggested improvements from previous assingment results:
"the treatment of leakage and refit strategy appears technically sound, but the written justification is quite thin. There is limited explicit discussion of exactly how leakage is/should be avoided or how the walk-forward refitting behaves in practice. Similarly, there is not much evidence of feature engineering beyond what is in the baseline: it would be good to see experiments or ablations with alternative feature sets, and some evidence/arguments about which features are most important or why"


Within this project, I will be performing the following:

#### 1. Data Preparation:
"Using the same set of tickers as in Part 1 (XLK, XLP, XLV, XLF, XLE, XLI) to construct a multivariate timeseries from daily or weekly prices or other derived features (such as returns and volatility)."

**For this assignment, I have decided upon using weekly adj_close prices. I used weekly data for clustering as:
- weekly data has less noise compared to daily data, 
- rolling stats can be more stable on weekly data, compared to more volatile daily data,
- less computational cost, leading to faster clustering

As mentioned in McGreevy et al's "Detecting multivariate market regimes via clustering algorithms" (2024),[1]  data is sized to aproximate weekly windows: "We assume that we want to trade over a specific window and take h1 = 35 equivalent to one market week in market hours"

Within this assignment, I use adj_close as it accounts for dividends, corporate actions and splits - which leads to better analysis for long-term investing.**

#### 2. Regime Detection:

"Apply a clustering algorithm of your choice (e.g. k-means, Gaussian Mixture Models, hierarchical clustering) To identify latent regimes (ie clusters) from this multivariate time series."

For this assignment, I will be attempting the clustering algorithm; K-Means.

#### 3. Integration with Prediction
"Use the detected regime labels as additional features in your predictive modelling task from Part 1 (the Student class with fit/predict).
• Evaluate whether regime-informed prediction improves over your Part 1 Student class as the baseline"

For this assignment, I will attempt to integrate my chosen clustering algorithm with my original part 1 baseline student.py and mltester. I will also run experiments to see if certain fields/methods, e.t.c. changed within my *Regime Detection* section can help improve the overall Directional Accuracy and decrease MAE and RMSE.

#### 4. Baseline Comparison

"In addition to your own Part 1 baseline, you must compare your results against at least one published baseline method" I will be using: "McGreevy, J., Muguruza, A., Issa, Z., Salvi, C., Chan, J., & Zuric, Z. (2024). Detecting Multivariate Market Regimes Via Clustering Algorithms. SSRN [http://dx.doi.org/10.2139/ssrn.4758243]"

## 1. Data Preparation

The first clustering algorithm I'm attempting is K-Means Clustering. K-means has been proven as a successful clustering algorithm in previous academic papers, such as "Horvath, Issa & Muguruza (2021); Clustering Market Regimes Using the Wasserstein Distance"[2]; found that using a variant of K-means (WK-means) got high accuracy in detecting market regimes in univariate time series.




In [3]:
%run DataPreparation/KMeans_Data_Prep




-------------------------------
K = 2
Silhouette: 0.334 | CH: 140.4

[INTERPRETATION - TRAIN]
Cluster 0: ret=0.0072, vol=0.0113 -> BULL
Cluster 1: ret=-0.0193, vol=0.0140 -> BEAR
Saved: detected_regimes_k2.csv

-------------------------------
K = 3
Silhouette: 0.303 | CH: 141.0

[INTERPRETATION - TRAIN]
Cluster 0: ret=0.0204, vol=0.0137 -> BULL
Cluster 1: ret=0.0035, vol=0.0109 -> NEUTRAL
Cluster 2: ret=-0.0300, vol=0.0141 -> BEAR
Saved: detected_regimes_k3.csv

-------------------------------
K = 4
Silhouette: 0.149 | CH: 120.1

[INTERPRETATION - TRAIN]
Cluster 0: ret=-0.0065, vol=0.0111 -> NEUTRAL
Cluster 1: ret=0.0192, vol=0.0142 -> NEUTRAL
Cluster 2: ret=-0.0353, vol=0.0157 -> BEAR
Cluster 3: ret=0.0116, vol=0.0106 -> BULL
Saved: detected_regimes_k4.csv

-----------------------------------------------------


Chicken and egg problem?


Within Appendix/DataPreparation/KMeans_Data_Prep.py; I loaded daily prices from prices.csv (specifically adj_close), normalised tickers and converted long format into a wide panel of the six sector ETFs. From this weekly panel, I created causal features including weekly log returns, 12-week rolling volatility and 6-week momentum for each ticker, along with market level aggregates (i.e. cross sectional volaitility) for capturing overall market conditions. I drop rows with incomplete features and then split the dataset into train and test sets based on a fixed date (I chose 4th Jan 2019 as it is a Friday, it provides 9 years of training data (2010-2018) and 5 years of testing data (2019-2023)) as to avoid look-ahead bias. 

ISLP[3] suggests "We strongly recommend always running K-means clustering with a large value of n_init, such as 20 or 50, since otherwise an undesirable local optimum may be obtained", hence why my n_init=50. Later in this assignment, I will trying n_init values of 20, 30, 40 and 50.

As a practical consideration, for standardising inputs, I use StandardScaler to minimise the within-cluster Euclidean distances. As mentioned on page 532 of ISLP [3], standardisation is preferred on K-means, so later in this assignment, I will be trying different scalers from the sklearn.preprocessing package to see which returns the best directional accuracy and lowest MAE and RMSE.

To understand the best K value for my k-means clustering on the prices.csv data, I've opted to use a mixture of silhouette score and Calinski-Harabasz scores. To combat data leakage, I've included the creation of the regime_lag1 so that predictiosn only use information available at prediction time. 


Interpreting the Data;
The terms; "Bull", "Neutral" and "Bear" are used to refer to stock market conditions - used to describe how the market is doing in general. According to Investopedia;( https://www.investopedia.com/insights/digging-deeper-bull-and-bear-markets/ ), "A bull market is a market that is on the rise and where the economy is sound. A bear market exists in an economy that is receding, where most stocks are declining in value." Neutral market indicates a market that is stable, which is neither rising, nor falling.

Within my data preparation file, I have a heuristic rule for the different K values.

For K=2, I use binary classification on returns, where if average return is posivtive it is a "Bull", whereas negative it is "Bear".

For K=3 and K=4, I use ternary classification on return and volatility, to create a 2x2 decision matrix:

| Return VS Median |  Volatility VS Median | Label    |
|----------        |---------             -|----------|
| Above Median     | Below Median          | Bull     |
| Above Median     | Above Median          | Neutral  |
| Below Median     | Below Median          | Neutral  |
| Below Median     | Above Median          | Bear     |

I use siholuette and Calinski-Harabasz to understand what would be the best K value. 

Use of Silhouette Score:
The silhouette score ranges from -1 to 1; "with the score being bounded between -1 and 1. [A positive value] indicates that the point is appropriately clustered, whereas values near 0 denote ambiguity, and negative values hint at a possible misclassification." according to NumberAnalytics ( https://www.numberanalytics.com/blog/silhouette-score-clustering-evaluation ).

Use of Calinski-Harabasz (CH):
The Calinski-Harabasz index (also known as variance ratio criterion) measures "how similar an object is to it's own cluster (cohesion) compared to other clusters (seperation)" according to GeeksForGeeks ( https://www.geeksforgeeks.org/machine-learning/calinski-harabasz-index-cluster-validity-indices-set-3/ ). A higher value for CH means that clusters are dense are well seperated. 

Results:
For K=2:
- Silhouette Score is 0.334, CH Index is 140.4
- Cluster 0 is bull
- Cluster 1 is bear

The silhouette score is the highest out of the different K values, indicating the clearest regime distinction. This is the most optimal k value as it's the simplest model for capturing the main market states.

For K=3:
-  Silhouette Score is 0.303, CH index is 141.0
- Cluster 0 is bull
- Cluster 1 is neutral
- Cluster 2 is bear

The silhouette score is lower than it is for k=2, but CH index is slightly higher. Silhouette score is more important for intrepability however, as Silhouette has a bounded range (-1 to 1) compared to CH, which has no bounded range. This k value captures netural periods (slightly positive returns, low volatility).

For K=4
- Silhouette Score is 0.149, CH index is 120.1

The Silhouette score drops dramatically to 0.149 and CH index also has the lowest value out of all the k values; indicating the worst seperation.

For this assignment, I will be using k=2.


### Integrating with Prediction:

Below is my baseline student from Coursework Part 1, saved as "student.py" which uses ElasticNet:

In [4]:
# Normal Student

from mltester import run_mltester
mean_diraccuracies_student = []
mean_mae_student = []
mean_rmse_student = []
horizons = [1, 5, 10, 20, 40, 50, 70, 100]

for horizon in horizons:
    results = run_mltester(
        model_spec="student.py:Student",  
        tickers=["XLE", "XLF", "XLI", "XLK", "XLP", "XLV"],            
        data_file="prices.csv",
        horizon=horizon,
        step=10,                           
    )
    mean_diraccuracy = results['diracc'].iloc[-1]  
    mean_diraccuracies_student.append(mean_diraccuracy)
    print(f"Horizon {horizon:2d} days: {mean_diraccuracy:.3f}")

    mean_mae = results['mae'].iloc[-1] 
    mean_mae_student.append(mean_mae)
    print(f"Mean MAE: {mean_mae:.3f}")

    mean_rmse = results['rmse'].iloc[-1]  
    mean_rmse_student.append(mean_rmse)
    print(f"Mean RMSE: {mean_rmse:.3f}")


   XLE: DirAcc=0.5072  MAE=0.011832  RMSE=0.017435
   XLF: DirAcc=0.5111  MAE=0.009444  RMSE=0.014018
   XLI: DirAcc=0.5334  MAE=0.008395  RMSE=0.012315
   XLK: DirAcc=0.5469  MAE=0.009224  RMSE=0.013457
   XLP: DirAcc=0.5270  MAE=0.005977  RMSE=0.008693
   XLV: DirAcc=0.5228  MAE=0.007140  RMSE=0.010181
Saved outputs to C:\Users\40261885\OneDrive - Queen's University Belfast\Desktop\MachineLearningAssignmentPt2\ClusteringAttempts\outputs
Horizon  1 days: 0.525
Mean MAE: 0.009
Mean RMSE: 0.013
   XLE: DirAcc=0.5186  MAE=0.026941  RMSE=0.039138
   XLF: DirAcc=0.5510  MAE=0.020891  RMSE=0.029593
   XLI: DirAcc=0.5727  MAE=0.019090  RMSE=0.027049
   XLK: DirAcc=0.5889  MAE=0.020054  RMSE=0.027495
   XLP: DirAcc=0.5640  MAE=0.012760  RMSE=0.017761
   XLV: DirAcc=0.5637  MAE=0.015727  RMSE=0.021495
Saved outputs to C:\Users\40261885\OneDrive - Queen's University Belfast\Desktop\MachineLearningAssignmentPt2\ClusteringAttempts\outputs
Horizon  5 days: 0.560
Mean MAE: 0.019
Mean RMSE: 0.027
  


To integrate my initial student.py from Coursework part 1 with Kmeans; I extended the class (please see student_ext.py) by creating the external file (via KMeans_Data_Prep.py file, "detected_regimes_k{K}.csv", e.g. detected_regimes_k2.csv) and making it required in my student_ext.py (line 28)

Within my extended student, I've made it so that the columns; date, regime and regime_lag1 are required (for no data leakage, lines 50-55), input is the X dataframe (same as the original code) and also my detected_regimes_k2.csv. DateTimeIndex is sorted because time series operations like .loc[date] and shift() assume chronological order (line 60).

For the _make_features() function in my extended student, it now has 13 features instead of 12 (original part 1 student features with regime column, line 179). Helper methods are same as my original part 1 student code (lines 93-121). 

My extended student has a new function; _get_regime_for_date(self, date) (lines 126-140); this is a time-safe regime lookup function using the lagged regime data (line 129), again to ensure no data leakage (by ensuring regime at time t uses information up to t-1) (lines 63, 64 and line 129) and includes some handling of edge cases, such as NaN values (line 134). 

For the fit() function, I've added one-hot encoding of regime labels (lines 200-203), creating regime-specific indicator features. Additionally, I store the training column strucutre to ensure consistent feature alignment during prediction (line 206).  

For predict() I have added column alignment logic (lines 278-288) to ensure prediction features match the training structure, including creating one-hot regime columns for new data and reordering columns to maintain consistency with training phase.

Compared to my original student, my extended student contains error-handling, such as validating that  the regime file has required columns (lines 50-56), handles regimes not seen during training (line 287), aligns prediction features to training structures and manages int64 nullable types (lines 68, 69, 200, 275) and is overall more robust than my original student.py class. 

Within my kmeans student class, I've also embedded the core functions of mltester; specifically forward_log_return(), compute_metrics(), walk_forward_predict(), evaluate_ticker(). Within my walk_forward_predict() (compared to the original mltester.py) I made it so that each walk-forward block gets a fresh model with the same regime file. I've also simplified data loading, compared to the original mltester.py (lines 396, 407).

Looking at the results of the given DirAcc, MAE and RMSE from different K values, it successfully demonstrates that regime detection works due to differenet results, and as expected, K=2 performs best as it has better DirAcc and lower MAE/RMSE on average compared to K=3 and K=4. 


In [ ]:
# trying all the differnt k's to verify silhouette/CH findings

### Experimentation of Different Methods within my KMeans clustering.

As of right now, my Kmeans clustering file uses;
- StandardScaler(),
- No PCA to generate clusters,
- n_init = 50,

In an attempt to improve the Directional Accuracy and decrease the MAE and RMSE values; I will try:
- changing the StandardScaler() to RobustScaler(),
- chanding the StandardScaler() to MinMaxScaler(),
- implement PCA within the cluster generation and 
- changing the n_init value to 20 (between 20-50, as per ISLP's guidance). 
- changing the n_init value to 30 


Below I've detailed my experimentation journey for each experiment method. 

#### RobustScaler (aka Robust):

Within my KMeans_Data_Prep, I changed the scaler used on line 42 from StandardScaler() to RobustScaler() (Appendix/Experimentation/KMeans_Data_Prep_robustScaler). And changed the scaler used in the Appendix/Experimentation/student_kmeans_multvariate_robust.py which runs the mltester functionality on my computed timeseries. I decided to try replacing StandardScaler() with RobustScaler() as StandardScaler() (according to GeeksForGeeks: https://www.geeksforgeeks.org/machine-learning/standardscaler-minmaxscaler-and-robustscaler-techniques-ml/ ), "subtracts the mean of the data and divides it by standard deviation. This centers the data around zero and standardizes the variablity". It is more sensitive to outliers. Whereas RobustScaler "subtracts the median of data and divides by interquartile range (IQR) which helps in reducing the effect of outliers why maintaining distribution of non-outlier values."

#### PCA:
Within my KMeans_Data_Prep, I've added PCA (principle component analysis) for dimensionality reduction and to help "reduce the number of features in a dataset while keeping the most important information"  according to GeeksForGeeks (https://www.geeksforgeeks.org/data-analysis/principal-component-analysis-pca/ ). This in itself can lead to less noise and redundant information, resulting in cleaner, more stable clusters. PCA also looks to resolve the curse of dimensionality , as in high-dimensional spaces data can become sparse and can lead to an overfitting risk.

#### n_init20 (aka Init20) and n_init30 (aka Init30)
Within my KMeans_Data_Prep, I've changed the n_init value from 50 to 20 and 30 by changing the n_init parameter within the kmeans initialisation (line 55).
The n_init parameter is defined as the "Number of times the k-means algorithm is run with different centroid seeds" according to the scikit-learn docs (https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html ). 


#### Directional Accuracy Results

![directional_accuracy_zoomed.png](method_comparison/directional_accuracy_zoomed1.png)
(The above graph is zoomed in for easier distinction between bars)



From the results; 
- Both the Init20 and Init30 have the same averages as each other across the differing k values, indicating that changing the n_init value does not make a difference to the directional accuracy.
- For PCA addtion in regime detection; it has a high DirAcc in k=2 and k=3, then second highest in k=4, but stil higher than my original kmeans code. 
- For RobustScaler; it is the lowest scoring method for DirAcc in k=2, the same as other  for k=3 but has the highest score for k=4.
- For my original student kmeans, it has a high DirAcc in k=2 and k=3, but then has the lowest DirAcc overall in K=4, indicating that my baseline clustering has become unstable at higher K values.

Worst performing was Robust in K=2, then for k=4 it was init20, init 30 and my kmeans student.

Overall I have found that regimes can improve predictional accuracy. For K=2, the highest directional accuracy is given by init20, init30, PCA and my original kmeans student. DirAcc is the same across all methods at k=3 and then Robust has the highest DirAcc at K=4.

#### MAE Results

![mae_zoomed.png](method_comparison/mae_zoomed1.png)
(The above graph is zoomed in for easier distinction between bars)

From the results:
- Similar to DirAcc, both the Init20 and Init30 have the same averages for MAE as each other for the differing K values, indicating that changing the n_init value does not make a difference to MAE score.
- PCA has the third highest MAE score for k=2, then the highest MAE for k=3, then second lowest for k=4.
- Robust has the lowest MAE score overall at all k values.
- My original kmeans student has the highest MAE at k=2, second highest at k=3 and has the same MAE value as init20 and init30 at k=4.

Overall, for minimising the MAE score, RobustScaler() performed best in all k values. Worst performing (as in, highest scoring) was my student kmeans in k=2, PCA for k=3, then init20, init30 and my student kmeans for k=4.

#### RMSE Results

![rmse_zoomed.png](method_comparison/rmse_zoomed1.png)
(The above graph is zoomed in for easier distinction between bars)

Results are very similar to results in MAE in regards to output. Overall, for minimising the RMSE score, RobustScaler() performed best in all k values. Worst performing (as in, highest scoring) was my student kmeans in k=2, PCA for k=3, then init20, init30 and my student kmeans for k=4.

#### Decisions made based on results

Based upon the results of my experimentation, I must consider the trade-off of having higher directional accuracy with higher MAE and RMSE, or slightly lower directional accuracy with lower MAE and RMSE. As RobustScaler() has shown to be stable throughout DirAcc, MAE and RMSE results, (especially considering the difference between the highest scoring methods for K=2 and Robust was 0.0008), I have decided to implement it within my student_ext.py. 

To run my student_ext.py, I will first prepare the data (now with using RobustScaler file)

In [5]:
%run DataPreparation/KMeans_Data_Prep_robustScaler


-------------------------------
K = 2
Silhouette: 0.334 | CH: 144.5

[INTERPRETATION - TRAIN]
Cluster 0: ret=0.0075, vol=0.0114 -> BULL
Cluster 1: ret=-0.0207, vol=0.0137 -> BEAR
Saved: detected_regimes_k2_robustscaler.csv

-------------------------------
K = 3
Silhouette: 0.296 | CH: 142.5

[INTERPRETATION - TRAIN]
Cluster 0: ret=0.0211, vol=0.0137 -> BULL
Cluster 1: ret=-0.0294, vol=0.0140 -> BEAR
Cluster 2: ret=0.0035, vol=0.0109 -> NEUTRAL
Saved: detected_regimes_k3_robustscaler.csv

-------------------------------
K = 4
Silhouette: 0.154 | CH: 122.5

[INTERPRETATION - TRAIN]
Cluster 0: ret=-0.0067, vol=0.0111 -> NEUTRAL
Cluster 1: ret=0.0195, vol=0.0141 -> NEUTRAL
Cluster 2: ret=0.0115, vol=0.0106 -> BULL
Cluster 3: ret=-0.0361, vol=0.0158 -> BEAR
Saved: detected_regimes_k4_robustscaler.csv

-----------------------------------------------------


In [ ]:
%run student_ext.py


TESTING K=2 REGIMES

Evaluating horizon 1 days...
   XLE: DirAcc=0.5127  MAE=0.011796  RMSE=0.017381
   XLF: DirAcc=0.5148  MAE=0.009424  RMSE=0.013988
   XLI: DirAcc=0.5376  MAE=0.008376  RMSE=0.012288
   XLK: DirAcc=0.5490  MAE=0.009201  RMSE=0.013440
   XLP: DirAcc=0.5323  MAE=0.005938  RMSE=0.008649
   XLV: DirAcc=0.5281  MAE=0.007118  RMSE=0.010159
Horizon   1 days: DirAcc=0.529 | MAE=0.0086 | RMSE=0.0127

Evaluating horizon 5 days...
   XLE: DirAcc=0.5305  MAE=0.026839  RMSE=0.039030
   XLF: DirAcc=0.5568  MAE=0.020812  RMSE=0.029535
   XLI: DirAcc=0.5807  MAE=0.019007  RMSE=0.026991
   XLK: DirAcc=0.5958  MAE=0.020012  RMSE=0.027484
   XLP: DirAcc=0.5791  MAE=0.012665  RMSE=0.017620
   XLV: DirAcc=0.5714  MAE=0.015699  RMSE=0.021462
Horizon   5 days: DirAcc=0.569 | MAE=0.0192 | RMSE=0.0270

Evaluating horizon 10 days...
   XLE: DirAcc=0.5349  MAE=0.038098  RMSE=0.055896
   XLF: DirAcc=0.5857  MAE=0.028969  RMSE=0.041096
   XLI: DirAcc=0.5937  MAE=0.026170  RMSE=0.037358
   XLK:

In [ ]:
%run PublishedComparison/ComputeRegimes_McGreevy.py

# this creates regimes_mcgreevy_weekly.csv

Saved: regimes_mcgreevy_weekly.csv


## 4. Baseline Comparison

For baseline comparison, I will be using the "Detecting Multivariate Market Regimes via Clustering Algorithms" method (2024) by James McGreevy (Aitor Muguruza-Gonzalez, Zacharia Issa Jonathan Chan and Cris Salvi).


Key takeaways of the McGreevy et al paper is as follows:

"• We develop an adapted k-means algorithm that uses the 2-Wasserstein distance metric or Maximum Mean Discrepancy, and d-dimensional data in order to identify changes in joint market
regimes between assets, in particular correlation.
• We create a two-step process for finding the marginal and joint market regimes in synthetic and
real data.
• Using the two-step process, we form approximations to the mean, variance and correlation which
then subsequently inform profitable portfolios of pairs of stocks."

To begin creating my version of McGreevy et al for this assignment, I first created a script to compute regimes, then also created the class containing the fit() and predict() functions that uses the computed regimes. For McGreevy et al (2024), the paper's main contribution is showing how it adapts clustering for the specific problem - using a probability metric (specifically; 2-Wasserstein distance between empirical measures) within it's clustering framework to compare the data segment distributions, rather than using raw data points. More specifically, the novelty of McGreevy et al's is replacing Euclidean distance with Wasserstein to compare data segment distributions.

### Creation of the Regimes (ComputeRegimes_McGreevy.py)


For McGreevy et al, it states: "Therefore,
we may rewrite the goal of the MRCP (market regime clustering problem) as being equivalent to assigning labels to empirical probability measures µ ∈ P_p(R^d), where P_p(R^d) is the set of probability measures on R^d with finite pth moment.
In practice we wish to take potentially overlapping segments of returns data for d assets and gather the empirical measures associated to these segments together into distinct clusters, each defined by an
underlying distribution, which we then refer to as market regimes."  For my preparation of the data, I use:

segs, seg_dates = segment(returns_z, H1, H2)


To segment the lenght of h1, overlapping h2.

![h1_minus_h2_block_mcgreevy.png](Images/h1_minus_h2_block_mcgreevy.png)

(Image is taken from another McGreevy et al paper (2023)).

Figure 5.2 is a horizontal timeline of returns divided into boxes (where each box = an observation, e.g. one week). h_1 represents the segment length (e.g. from the image; 5 boxes = 5 weeks). h_2 represents the overlap between consecutive segments (e.g. spanning all 5 boxes / weeks). The red block indicates the current segment being clustered. The segments (\mu_1, \mu_2, e.t.c.) represents an empirical distribution of returns over h_1 weeks, with the lower arrows showing an overlapping sliding forward windows by h_1 - h_2 steps. I attempt to recreate this in my segment function.

Within my segment function, I create overlapping windows of length h1 (20 weeks in my configuration). For my segments; each new segment starts h1-h2 weeks after previous one (20-16= 4); where h1 is the full segment and h2 is the overlap. I create a matrix of returns for six tickers over h1. The dates variable acts as an anchor for the next interval. 

For preprocessing, I chose weekly resampling, to keep it in line with my current data preprocessing for this assignment. I create a wide panel (1 column per ticker) to implement the multivariate setting, using k=2 as mentioned in the McGreevy paper: "The simplest possible choice is k = 2", and seed kept at 42 to keep it the same throughout my assignment. 


Within my code (appendix/PublishedComparision/ComputeRegimes_McGreevy.py), I have my segment function:

"def segment(R, h1, h2):
    step = h1 - h2
    segs = [R.iloc[i-h1+1:i+1].values for i in range(h1-1, len(R)-1, step)]
    dates = [R.index[i+1] for i in range(h1-1, len(R)-1, step)]
    return segs, pd.DatetimeIndex(dates)"

The purpose of this segment function is to create the foundation of the published method; specifically by dividing the multivariate timeseries into overlapping rolling windows for clustering. Within this I use the parameters H1 and H2



![h1_minus_h2_block_mcgreevy.png](Images/h1_minus_h2_block_mcgreevy.png)



-----------------------------------------------
def w2(X, Y):
    if X.shape[0] != Y.shape[0]:
        m = min(X.shape[0], Y.shape[0])
        X, Y = X[:m], Y[:m]
    
    if X.shape[1] == 1:
        return np.sqrt(np.mean((np.sort(X[:, 0]) - np.sort(Y[:, 0]))**2))
    
    C = np.sum((X[:, None, :] - Y[None, :, :])**2, axis=2)
    r, c = linear_sum_assignment(C)
    return np.sqrt(C[r, c].mean())

----------------------------------------

For my w2(X, Y) function, it is the Wasserstein-2 distance between empiracl measures. I use Wasserstein as the paper emphasises it's use as a core metric for computing the distance between 2 empirical distributions, e.g. in the Abstract: "In particular, we empirically investigate the performance of the algorithm endowed with either Wasserstein distances or Maximum Mean Discrepancies". Overall, the function w2(X,Y) implements the 2-Wasserstein distance between the two empirical segment distributions (X and Y). The Wasserstein distance is in 1-D using the quantile closed-form (Section 4.1.1 [1]). The 'C' variable builds the cost matrix of squared Euclidean distances. THe 'r, c' variables run the Hungarian algorithm on 'C' (the linear_sum_assignmnet), and then returns the final distance by taking the square root of the mean of the optimal assignment costs.

For my wkmeans(D, k, prev_centres=None, seed=42) function, I run a kmeans style loop on a precomputed distance matrix (D), which is entries of 2-Wasserstein distances between empirical measures (segments). I then assign each segment to it's closest centre under 2-Wasserstein. 

I then update the step (barycentre approximation: " 

intra = D[np.ix_(idx, idx)].sum(1)
new_centres.append(int(idx[int(np.argmin(intra))]))"
                
, as per paper: "Furthermore, the concept of an average or central measure amongst a set of measures has a natural form when using the p-Wasserstein distance. This is called the Wasserstein barycentre") for each cluster - choosing the new centre as the medoid. The medoid is defined as "the most centrally located data point within a cluster." According to GeeksForGeeks (https://www.geeksforgeeks.org/machine-learning/k-medoids-clustering-in-machine-learning/ )

Although specifically not detailed within McGreevy et al, I included empty cluster handling ("if len(idx) == 0: ..." - if idx is empty, there are no members in the cluster, my if statement reseeds the centre so the algorithm can continue). 

Mentioned within the McGreevy baseline is the two-step Copula variant: "Step 2: Multivariate clustering
The joint distribution of our newly transformed dataset should be the copula of the original data, and so we apply the 2-d WK-means or 2-d MMDK-means algorithm to our transformed data in order to obtain correlation regimes" . I was not able to code this due to it's complexity, so my ComputeRegimes_McGreevy.py includes the core single-stage clustering piece withn my wkmeans().

Also within my wkmeans()  I implement label persistence by building a cost matrix between previous and current centres, finding the minimum cost between old and new centres (Hungarian), creating a remap from new centre indext to old centre index, remap point labels so they refer to the same 'identity' as previous iteration and reorder centres to match previous identity order:

    if prev_centres is not None:
        cost = D[np.ix_(prev_centres, centres)]
        r, c = linear_sum_assignment(cost)
        remap = {new: old for old, new in zip(r, c)}
        labels = np.array([remap.get(L, L) for L in labels], dtype=int)
        centres = centres[c]
    
    return labels, centres






In [ ]:

from mltester import run_mltester

mean_diraccuracies_mcgreevy = []
mean_mae_mcgreevy = []
mean_rmse_mcgreevy = []
horizons = [1, 5, 10, 20, 40, 50, 70, 100]

for horizon in horizons:
    results = run_mltester(
        model_spec="PublishedComparison/mcgreevy_baseline.py:McGreevyBaseline",
        tickers=["XLE", "XLF", "XLI", "XLK", "XLP", "XLV"],
        data_file="prices.csv",
        horizon=horizon,
        step=10,
    )
    diracc = results['diracc'].iloc[-1]
    mae    = results['mae'].iloc[-1]
    rmse   = results['rmse'].iloc[-1]

    mean_diraccuracies_mcgreevy.append(diracc)
    mean_mae_mcgreevy.append(mae)
    mean_rmse_mcgreevy.append(rmse)

    print(f"Horizon {horizon:3d} days: DirAcc={diracc:.3f} | MAE={mae:.4f} | RMSE={rmse:.4f}")


   XLE: DirAcc=0.4828  MAE=0.011843  RMSE=0.017445
   XLF: DirAcc=0.4910  MAE=0.009461  RMSE=0.014030
   XLI: DirAcc=0.5130  MAE=0.008410  RMSE=0.012329
   XLK: DirAcc=0.5239  MAE=0.009236  RMSE=0.013464
   XLP: DirAcc=0.5095  MAE=0.005978  RMSE=0.008695
   XLV: DirAcc=0.5141  MAE=0.007143  RMSE=0.010195
Saved outputs to C:\Users\40261885\OneDrive - Queen's University Belfast\Desktop\MachineLearningAssignmentPt2\ClusteringAttempts\outputs
Horizon   1 days: DirAcc=0.506 | MAE=0.0087 | RMSE=0.0127
   XLE: DirAcc=0.4841  MAE=0.026992  RMSE=0.039153
   XLF: DirAcc=0.5080  MAE=0.021022  RMSE=0.029708
   XLI: DirAcc=0.5422  MAE=0.019263  RMSE=0.027131
   XLK: DirAcc=0.5653  MAE=0.020094  RMSE=0.027505
   XLP: DirAcc=0.5507  MAE=0.012726  RMSE=0.017750
   XLV: DirAcc=0.5547  MAE=0.015713  RMSE=0.021542
Saved outputs to C:\Users\40261885\OneDrive - Queen's University Belfast\Desktop\MachineLearningAssignmentPt2\ClusteringAttempts\outputs
Horizon   5 days: DirAcc=0.534 | MAE=0.0193 | RMSE=0.027

References:
[1] Detecting Multivariate Market Regimes via Clustering Algorithms, James Mc Greevy, Aitor Muguruza Gonzalez, Zacharia Issa, Jonathan Chan, Cris Salvi (2022-2023) Imperial College London


[2] Horvath, Issa & Muguruza (2021); Clustering Market Regimes Using the Wasserstein Distance


[3] An Introduction to Statisitcal Learning; Gareth James, Daniela Witten, Trevor Hastie, Robert Tibshirani, Johnathan Taylor (2023)
